In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns

from timeseries import plot_ts

In [2]:
data = pd.read_parquet("../data/03_primary/features.pq")

In [4]:
data.describe()

,VehPower,VehAge,DrivAge,BonusMalus,Density
count,678005.000000,678005.000000,678005.000000,678005.000000,678005.000000
mean,6.454656,7.044208,45.499020,59.761617,1792.428023
std,2.050903,5.666239,14.137486,15.636714,3958.667052
min,4.000000,0.000000,18.000000,50.000000,1.000000
25%,5.000000,2.000000,34.000000,50.000000,92.000000
50%,6.000000,6.000000,44.000000,50.000000,393.000000
75%,7.000000,11.000000,55.000000,64.000000,1658.000000
max,15.000000,100.000000,100.000000,230.000000,27000.000000


In [5]:
data.head()

,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Area,Density,Region
IDpol,,,,,,,,,
1,5,0,55,50,B12,Regular,D,1217,Rhone-Alpes
3,5,0,55,50,B12,Regular,D,1217,Rhone-Alpes
5,6,2,52,50,B12,Diesel,B,54,Picardie
10,7,0,46,50,B12,Diesel,B,76,Aquitaine
11,7,0,46,50,B12,Diesel,B,76,Aquitaine


In [10]:
data.VehGas.value_counts()

VehGas
Regular    345870
Diesel     332135
Name: count, dtype: int64

In [11]:
data.VehPower.value_counts()

VehPower
6     148976
7     145400
5     124821
4     115342
8      46956
10     31354
9      30085
11     18352
12      8214
13      3229
15      2926
14      2350
Name: count, dtype: int64

In [13]:
data.VehBrand.value_counts()

VehBrand
B12    166024
B1     162729
B2     159860
B3      53395
B5      34753
B6      28548
B4      25179
B10     17707
B11     13585
B13     12178
B14      4047
Name: count, dtype: int64

In [14]:
data.Area.value_counts()

Area
C    191880
D    151589
E    137166
A    103957
B     75459
F     17954
Name: count, dtype: int64

In [15]:
data.Region.value_counts()

Region
Centre                         160601
Rhone-Alpes                     84752
Provence-Alpes-Cotes-D'Azur     79314
Ile-de-France                   69791
Bretagne                        42122
Nord-Pas-de-Calais              40275
Pays-de-la-Loire                38751
Languedoc-Roussillon            35798
Aquitaine                       31329
Poitou-Charentes                19046
Midi-Pyrenees                   17141
Basse-Normandie                 10893
Bourgogne                       10492
Haute-Normandie                  8784
Picardie                         7994
Auvergne                         5287
Limousin                         4567
Corse                            4516
Champagne-Ardenne                3026
Alsace                           2200
Franche-Comte                    1326
Name: count, dtype: int64